In [2]:
from ultralytics import YOLO

# Load a model
model_path="/work/csr33/object_detection/runs/detect/baseline_train/weights/best.pt"
model = YOLO(model_path)  # load a custom model

# Validate the model
metrics = model.val(split="test")  # no arguments needed, dataset and settings remembered
 # a list contains map50-95 of each category

Ultralytics YOLOv8.1.9 🚀 Python-3.11.6 torch-2.1.2 CUDA:0 (NVIDIA RTX 5000 Ada Generation, 32157MiB)
Model summary (fused): 268 layers, 68130309 parameters, 0 gradients, 257.4 GFLOPs


val: Scanning /work/csr33/ast/datasets/ast_yolov5/labels/test.cache... 4146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4146/4146 [00:00<?, ?it/s]
/hpc/group/borsuklab/cred/.conda/envs/yolov8/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 260/260 [01:01<00:00,  4.22it/s]


                   all       4146      21499      0.777      0.742      0.787        0.6
    sedimentation_tank       4146        812      0.801      0.889      0.906      0.757
           water_tower       4146        222      0.898      0.887      0.909      0.678
        spherical_tank       4146        252       0.82      0.774      0.804        0.6
      closed_roof_tank       4146      11009      0.864      0.914      0.954      0.765
external_floating_roof_tank       4146       1446      0.848      0.909       0.94      0.827
narrow_closed_roof_tank       4146       7639      0.755      0.684       0.79      0.392
      undefined_object       4146        119      0.451      0.134      0.207       0.18
Speed: 0.1ms preprocess, 12.0ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /work/csr33/object_detection/runs/detect/val10


In [4]:
metrics.box.map    # map50-95

0.599900218572653

In [5]:
metrics.box.map50  # map50


0.7871486420777859

In [9]:
metrics.box.f1

array([     0.8427,     0.89257,     0.79626,     0.88845,      0.8779,     0.71797,     0.20712])

In [23]:
metrics.confusion_matrix.matrix

array([[        721,           0,           0,          30,          73,           1,          19,         151],
       [          0,         198,           4,          10,           0,           0,           0,          17],
       [          0,           5,         191,          32,           0,           0,           0,          31],
       [          9,          11,          53,       10077,          42,        1012,          27,        1499],
       [         57,           0,           0,          96,        1307,          11,          21,         170],
       [          0,           0,           1,         561,           3,        5741,          15,        2852],
       [          5,           0,           0,           6,           3,           0,          17,          34],
       [         20,           8,           3,         197,          18,         874,          20,           0]])

In [19]:
import numpy as np

In [22]:
accuracy = np.trace(metrics.confusion_matrix.matrix) / np.sum(metrics.confusion_matrix.matrix)
#metrics.confusion_matrix.nc
#metrics.confusion_matrix.conf
#metrics.confusion_matrix.iou_thres  

0.695234830305108

In [ ]:
download_all_files(subset_naip_in_slosh.iloc[:100], tile_dir, connections=6)

In [2]:
import geopandas as gpd

In [3]:
naip_df = gpd.read_parquet("/work/csr33/images_for_predictions/naip_tile_in_slosh_modeled_area.parquet")

In [5]:
chunked_naip_data_dir="/work/csr33/images_for_predictions/chunked_naip_data"
chunked_naip_data_filename="chunked_naip_data"

In [6]:
# Calculate the number of rows per chunk
rows_per_chunk = len(naip_df) // 100
df_chunks = [naip_df.iloc[i : i + rows_per_chunk] for i in range(0, len(naip_df), rows_per_chunk)]
for i, chunk in enumerate(df_chunks):
    file_path = f'{chunked_naip_data_dir}/{chunked_naip_data_filename}_{i}.parquet'
    chunk.to_parquet(file_path, index=False)

In [32]:
tile_paths

['/work/csr33/images_for_predictions/naip_tiles/m_4307033_nw_19_060_20211007.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207115_sw_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4307140_sw_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207001_ne_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207116_se_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207001_se_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207108_nw_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4307148_nw_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4307148_ne_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4207106_sw_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4307147_se_19_060_20211024.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_4307057_sw_19_060_20211024.tif',
 '/w

In [27]:
chip_tiles_concurrent(tile_dir, img_dir, item_dim=640, connections=6)

<class 'AttributeError'>


10

In [40]:
def get_args_parse():
    parser = argparse.ArgumentParser("")
    parser.add_argument("--chunked_naip_data_dir", default="/work/csr33/images_for_predictions/chunked_naip_data", type=str)
    parser.add_argument("--chunked_naip_data_filename", default="chunked_naip_data", type=str)
    parser.add_argument("--chunk_id",  type=int)

    
    parser.add_argument("--backoff_factor", default=0.5, type=float)
    parser.add_argument("--max_retries", default=5, type=int)
    
    parser.add_argument("--processing_naip_dir", default="/work/csr33/images_for_predictions/processed_naip_data", type=str)
    parser.add_argument("--processing_naip_filename", default="processed_naip_data", type=str)
    parser.add_argument("--imgsz", default=640, type=int)

    args = parser.parse_args()
    return args
import sys
sys.argv = ['my_notebook']
args = get_args_parse()

In [53]:
#os.chdir("/work/csr33/object_detection")
#make sure processing naip dir exist

os.makedirs(args.processing_naip_dir, exist_ok=True)
naip_df = gpd.read_parquet("/work/csr33/images_for_predictions/chunked_naip_data/chunked_naip_data_96.parquet")
args.chunk_id = 96
naip_df["img_url"] = naip_df.assets.apply(lambda asset: asset["image"]["href"])#
data_processing_over_chunk(naip_df, args)

0 Execution time: 101.11169767379761 seconds
1 Execution time: 125.7008740901947 seconds
2 Execution time: 126.36712956428528 seconds
3 Execution time: 136.74841904640198 seconds
4 Execution time: 147.03376173973083 seconds
5 Execution time: 134.94598197937012 seconds
6 Execution time: 149.9262683391571 seconds
7 Execution time: 145.56880497932434 seconds



KeyboardInterrupt



In [2]:
naip_data_path="/work/csr33/images_for_predictions/naip_tile_in_slosh_modeled_area.parquet"

    #os.chdir("/work/csr33/object_detection")
    #make sure processing naip dir exist
import geopandas as gpd  
subset_naip_in_slosh = gpd.read_parquet(naip_data_path)

In [3]:
len(subset_naip_in_slosh)

14012

In [2]:
s = rasterio.open("https://naipeuwest.blob.core.windows.net/naip/v002/al/2021/al_030cm_2021/30087/m_3008709_ne_16_030_20211112.tif")
d = s.read()

<open DatasetReader name='https://naipeuwest.blob.core.windows.net/naip/v002/al/2021/al_030cm_2021/30087/m_3008709_ne_16_030_20211112.tif' mode='r'>